In [2]:
%pip install ultralytics



Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from ultralytics import YOLO
import glob

In [4]:
INPUT_DIR = r"C:/Users/user/Documents/AI_project_clone/data_set_helmet/css-data"
WORK_DIR = r"C:/Users/user/Documents/AI_project_clone//working"

In [4]:
import os

# 데이터 경로 설정
LABELS_DIR = os.path.join(INPUT_DIR, 'train/labels')  # train 라벨 디렉토리
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'person'}  # 유지할 클래스 ID
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑

def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)

# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")


라벨 파일 수정이 완료되었습니다.


In [5]:


# WORK_DIR 디렉토리 생성
os.makedirs(WORK_DIR, exist_ok=True)

# 수정된 클래스 설정
num_classes = 3
classes = ['helmet', 'No-helmet', 'Person']  # Hardhat -> helmet, NO-Hardhat -> No-helmet

# data.yaml 파일 생성
dict_file = {
    'train': os.path.join(INPUT_DIR, 'train/images'),
    'val': os.path.join(INPUT_DIR, 'valid/images'),
    'test': os.path.join(INPUT_DIR, 'test/images'),
    'nc': num_classes,
    'names': classes
}

# data.yaml 저장
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'w+') as file:
    yaml.dump(dict_file, file)

print(f"'data.yaml' 파일이 '{data_yaml_path}'에 저장되었습니다.")

# 라벨 데이터 수정 (Hardhat -> helmet, NO-Hardhat -> No-helmet, Person 그대로, 나머지 제거)
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'Person'}  # 기존 클래스 ID 매핑
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑


def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)


# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")

# 데이터셋 크기 출력
for mode in ['train', 'valid', 'test']:
    files = glob.glob(os.path.join(INPUT_DIR, mode, 'images', '*'))
    print(f'{mode} set size: {len(files)}\n')

# YOLOv8 모델 학습
model = YOLO('yolov8n.pt')

model.train(
    data=data_yaml_path,
    task='detect',
    imgsz=640,
    epochs=30,
    batch=16,
    mode='train',
    name='yolov8n_v1_train'
)

print("Training completed!")


'data.yaml' 파일이 'C:/Users/user/Documents/AI_project_clone//working\data.yaml'에 저장되었습니다.
라벨 파일 수정이 완료되었습니다.
train set size: 2605

valid set size: 114

test set size: 82

New https://pypi.org/project/ultralytics/8.3.63 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/user/Documents/AI_project_clone//working\data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v1_train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7

train: Scanning C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\train\labels... 2605 images, 1291 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:06<00:00, 404.28it/s]


train: New cache created: C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\train\labels.cache


val: Scanning C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\valid\labels... 114 images, 72 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<00:00, 287.30it/s]

val: New cache created: C:\Users\user\Documents\AI_project_clone\data_set_helmet\css-data\valid\labels.cache


...

      30/30      2.32G      1.126     0.7436       1.04         22        640: 100%|██████████| 163/163 [00:58<00:00,  2.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.02it/s]

                   all        114         79      0.949      0.747      0.855      0.508



30 epochs completed in 0.506 hours.
Optimizer stripped from runs\detect\yolov8n_v1_train5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolov8n_v1_train5\weights\best.pt, 6.2MB

Validating runs\detect\yolov8n_v1_train5\weights\best.pt...
Ultralytics 8.3.62  Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]


                   all        114         79      0.949      0.747      0.854      0.507
                helmet         42         79      0.949      0.747      0.854      0.507
Speed: 0.5ms preprocess, 6.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\detect\yolov8n_v1_train5
Training completed!


In [13]:
from ultralytics import YOLO
import cv2

# 학습된 모델 로드
model = YOLO('runs/detect/yolov8n_v1_train/weights/best.pt')  # 학습 완료된 모델 경로

# OpenCV를 사용하여 웹캠 열기
cap = cv2.VideoCapture(0)  # 0은 기본 카메라. 다른 카메라를 사용하려면 1 또는 다른 번호로 변경

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 카메라에서 실시간 객체 탐지
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # YOLO 모델로 프레임 추론
    results = model.predict(source=frame, save=False, show=False)

    # 탐지 결과를 그리기
    annotated_frame = results[0].plot()  # 결과를 그린 프레임

    # 화면에 표시
    cv2.imshow('YOLOv8 Real-time Detection', annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 helmet, 1 No-helmet, 18.2ms
Speed: 0.0ms preprocess, 18.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 1 No-helmet, 17.3ms
Speed: 1.0ms preprocess, 17.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 3 No-helmets, 18.2ms
Speed: 4.5ms preprocess, 18.2ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 13.4ms
Speed: 3.0ms preprocess, 13.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.4ms
Speed: 3.0ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.4ms
Speed: 5.1ms preprocess, 10.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 2 No-helmets, 10.2ms
Speed: 1.8ms preprocess, 10.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 helmet, 